# UMAP-Spatial-Heatmap v0.3.1

In [ ]:
%%capture
import sys
!{sys.executable} -m pip -q install scanpy
!{sys.executable} -m pip -q install leidenalg
!{sys.executable} -m pip -q install observable_jupyter
!{sys.executable} -m pip -q install clustergrammer2
!{sys.executable} -m pip -q install numpy==1.19.5

import numpy as np
import pandas as pd
import scanpy as sc
from matplotlib import pyplot as plt
from observable_jupyter import embed
from clustergrammer2 import net, Network, CGM2
from copy import deepcopy
import json
import zlib, json, base64

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; maxHeight: 5000}</style>"))

In [ ]:
def json_zip(j):

    zip_json_string = base64.b64encode(
        zlib.compress(
            json.dumps(j).encode('utf-8')
        )
    ).decode('ascii')

    return zip_json_string

# Dataset Path

In [ ]:
# dataset_path = 's3://vz-analyzed-merfish/MsBrain_VS32_Brain8_YC_04-02-2022/'
# dataset_path = 's3://vz-analyzed-merfish/MsBrain_VS32_Brain9_V1_YC_04-20-2022/'

# latest dataset for Colles
dataset_path = 's3://vz-analyzed-merfish/MsBrain_VS34_Brain2a_V6_YC_04-28-2022/'
# dataset_path = 's3://vz-analyzed-merfish/MsBrain_VS32_Brain3_V6_YC_04-14-2022/'

## Load Data
Check that you have permissions to read directly from S3. If so run the commands below. If not load the data from EFS (after having copied it). 

In [ ]:
!aws s3 ls {dataset_path}

In [ ]:
region_of_interest = 'region_1'

In [ ]:
!aws s3 ls {dataset_path}ExportPartitionedBarcodes/{region_of_interest}/

### Load Cell by Gene Matrix

In [ ]:
file_path = 'ExportPartitionedBarcodes/region_0/barcodes_per_feature.csv'
cell_by_gene = pd.read_csv(dataset_path + file_path, index_col=0)
cells = cell_by_gene.index.tolist()
cell_by_gene.shape

In [ ]:
keep_genes = [x for x in cell_by_gene.columns.tolist() if 'Blank' not in x]
len(keep_genes)
cell_by_gene = cell_by_gene[keep_genes]
cell_by_gene.shape

In [ ]:
cell_by_gene.index = range(len(cell_by_gene.index.tolist()))
gex_int = cell_by_gene.astype(np.int)
gex_int

### Compressed Sparse GEX

In [ ]:
gex_dict = {}
for inst_gene in gex_int.columns.tolist():
    if 'Blank' not in inst_gene:
        ser_gene = gex_int[inst_gene]
        ser_gene = ser_gene[ser_gene > 0]
        ser_gene = ser_gene.astype(np.int8)    
        gex_dict[inst_gene] = ser_gene.to_dict()

### Load Cell Metadata

In [ ]:
!aws s3 ls {dataset_path}ExportCellMetadata/region_0/feature_metadata.csv

In [ ]:
file_path = 'ExportCellMetadata/region_0/feature_metadata.csv'
meta_cell = pd.read_csv(dataset_path + file_path, index_col=0)
meta_cell = meta_cell.loc[cells]
meta_cell.index = range(len(meta_cell.index.tolist()))

# add barcode count to metadata
meta_cell['barcodeCount'] = cell_by_gene.sum(axis=1)

# initialize meta_gene
meta_gene = pd.DataFrame(index=cell_by_gene.columns.tolist())

meta_cell.shape

In [ ]:
meta_cell.head()

In [ ]:
meta_cell.volume.hist(bins=100, range=[0,500])

In [ ]:
min_barcode = 5
mean_barcodes = meta_cell[meta_cell.barcodeCount >= min_barcode].barcodeCount.mean()
print(mean_barcodes)

meta_cell.barcodeCount.hist(bins=100, range=[0,100])

### Filter Cells

In [ ]:
min_volume = 100
max_volume = 2000
min_barcode = 20

# filter cells based on volume and transcript count
select_cells = meta_cell[(meta_cell.volume > min_volume) & 
                           (meta_cell.barcodeCount > min_barcode) & 
                           (meta_cell.volume < max_volume)].index.tolist()

# save to anndata object
ad_viz_ini = sc.AnnData(X=cell_by_gene.loc[select_cells].values, 
                    obs=meta_cell.loc[select_cells], var=meta_gene)

ad_viz_ini

### UMAP and Leiden Cluster Data 

In [ ]:
%%capture
n_neighbors = 15
resolution = 1.0

# Leiden Clustering
######################
ad_viz = deepcopy(ad_viz_ini.copy())
sc.pp.normalize_total(ad_viz)
sc.pp.log1p(ad_viz)
#sc.pp.highly_variable_genes(ad_viz)
sc.tl.pca(ad_viz, svd_solver='arpack')
sc.pp.neighbors(ad_viz, n_neighbors=n_neighbors)
sc.tl.umap(ad_viz)
sc.tl.leiden(ad_viz, resolution=resolution)

Get category colors from Scanpy

In [ ]:
%%capture
# generate colors for categories by plotting
sc.pl.umap(ad_viz, color="leiden", legend_loc='on data')
cats = ad_viz.obs['leiden'].cat.categories.tolist()
colors = list(ad_viz.uns['leiden_colors'])
cat_colors = dict(zip(cats, colors))

In [ ]:
df_pos = ad_viz.obs[['center_x', 'center_y', 'leiden']].round(2)
df_pos.columns = ['x', 'y', 'leiden']
df_pos['y'] = -df_pos['y']
df_umap = ad_viz.obsm.to_df()[['X_umap1', 'X_umap2']].round(2)
df_umap.columns = ['umap-x', 'umap-y']

df_name = pd.DataFrame(df_pos.index.tolist(), index=df_pos.index.tolist(), columns=['name'])
df_obs = pd.concat([df_name, df_pos, df_umap], axis=1)
data = df_obs.to_dict('records')

### Leiden Gene Expression Heatmap

In [ ]:
%%capture
ser_counts = ad_viz.obs['leiden'].value_counts()
ser_counts.name = 'cell counts'
meta_leiden = pd.DataFrame(ser_counts)

cat_name = 'leiden'
sig_leiden = pd.DataFrame(columns=ad_viz.var_names, index=ad_viz.obs[cat_name].cat.categories)                                                                                                 
for clust in ad_viz.obs[cat_name].cat.categories: 
    sig_leiden.loc[clust] = ad_viz[ad_viz.obs[cat_name].isin([clust]),:].X.mean(0)
sig_leiden = sig_leiden.transpose()
leiden_clusters = ['Leiden-' + str(x) for x in sig_leiden.columns.tolist()]
sig_leiden.columns = leiden_clusters
meta_leiden.index = sig_leiden.columns.tolist()
meta_leiden['leiden'] = pd.Series(meta_leiden.index.tolist(), index=meta_leiden.index.tolist())

# colors for clustergrammer2
ser_color = pd.Series(cat_colors)
ser_color.name = 'color'
df_colors = pd.DataFrame(ser_color)
df_colors.index = ['Leiden-' + str(x) for x in df_colors.index.tolist()]

net = Network(CGM2)
net.load_df(sig_leiden, meta_col=meta_leiden, col_cats=['leiden', 'cell counts'])
net.filter_threshold(0.01, axis='row')
net.normalize(axis='row', norm_type='zscore')
net.set_global_cat_colors(df_colors)
net.cluster()

### Compile Zipped Data for Observable

In [ ]:
obs_data = {
    'gex_dict': gex_dict,
    'data': data, 
    'cat_colors': cat_colors,
    'network': net.viz    
}

zip_obs_data = json_zip(obs_data)
len(zip_obs_data)

# UMAP-Spatial-Heatmap

In [ ]:
url = '@vizgen/umap-spatial-heatmap-single-cell-0-3-0'
inputs = {
    'zoom': -3.75, 
    'ini_cat': 'leiden',
    'zip_obs_data': zip_obs_data
}
embed(url, cells=['viewof cgm', 'dashboard'], inputs=inputs, display_logo=False) 